In [1]:
import scala.io.Source
import scala.util.Try
import org.apache.spark.SparkContext
import org.apache.spark.ml.feature.{HashingTF, IDF, Tokenizer,StopWordsRemover}
import org.apache.spark.ml.linalg._
import org.apache.spark.mllib.linalg.distributed.{IndexedRow, IndexedRowMatrix, RowMatrix}
import org.apache.spark.sql.{SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql._
import org.apache.spark.rdd.RDD
import org.apache.spark.SparkConf
import org.apache.log4j.{Logger, Level}

In [2]:
val conf = new SparkConf().setAppName("spark-test").setMaster("local[*]")
val sc = new SparkContext(conf)
val spark = SparkSession.builder().config(sc.getConf).getOrCreate()

Name: org.apache.spark.SparkException
Message: Only one SparkContext may be running in this JVM (see SPARK-2243). To ignore this error, set spark.driver.allowMultipleContexts = true. The currently running SparkContext was created at:
org.apache.spark.SparkContext.<init>(SparkContext.scala:76)
$line14.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(<console>:46)
$line14.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(<console>:54)
$line14.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(<console>:56)
$line14.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(<console>:58)
$line14.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(<console>:60)
$line14.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(<console>:62)
$line14.$read$$iw$$iw$$iw$$iw$$iw$$iw.<init>(<console>:64)
$line14.$read$$iw$$iw$$iw$$iw$$iw.<init>(<console>:66)
$line14.$read$$iw$$iw$$iw$$iw.<init>(<console>:68)
$line14.$read$$iw$$iw$$iw.<init>(<console>:70)
$line14.$read$$iw$$iw.<init>(<console>:72)
$line14.$read

In [57]:
val df = spark.read.json("DO_record_per_line.json")//.json("file.json")
import spark.implicits._
val film = df.filter('id === 3879) //'id === 3879 or 
  .select('lang, 'id, 'name, regexp_replace('desc,lit("""[\p{Punct}]"""), lit(" ")).alias("desc"))

val filmLang = film.collect().map(r=>r(0)).toList.head

df = [cat: string, desc: string ... 4 more fields]
film = [lang: string, id: bigint ... 2 more fields]
filmLang = es


es

In [58]:
val corpus = df.filter('lang === filmLang)
  .select('id, 'name, regexp_replace('desc,lit("""[\p{Punct}]"""), lit(" ")).alias("desc"))
corpus.show()

+----+--------------------+--------------------+
|  id|                name|                desc|
+----+--------------------+--------------------+
|  59|El ABC del empren...|A través de difer...|
| 124|Pensamiento Cient...|Aprenderemos cómo...|
| 160|   Ser más creativos|¡Claro que todos ...|
| 166|Conceptos y Herra...|Este curso provee...|
| 196|Egiptología (Egyp...|Este curso introd...|
| 198|Continuidad y des...|¿Cuál es la impor...|
| 252|Histología básica...|Este curso está d...|
| 272|Desarrollo rápido...|Entender las meto...|
| 273|Innovación educat...|Este curso promue...|
| 386|        Pre-Calculus|Curso diseñado pa...|
| 387|Matemáticas y Mov...|El curso propone ...|
| 468|Tecnologías de in...|Analizar diferent...|
|3660|Evaluación de Dec...| The Course Este ...|
|3810|Diseña sitios web...| Aprende a crear ...|
|3870|Apps de iOS y And...| Crea las apps qu...|
|3872|Conozca las escal...| Desarrolle sus h...|
|3873|Realice acompañam...| Aprenda a utiliz...|
|3874|Conozca las es

corpus = [id: bigint, name: string ... 1 more field]


[id: bigint, name: string ... 1 more field]

In [59]:
val tokenizer = new Tokenizer()
  .setInputCol("desc")
  .setOutputCol("wordsR")
val wordsDataRaw = tokenizer.transform(corpus)
val remover = new StopWordsRemover()
  .setInputCol("wordsR")
  .setOutputCol("words")
val wordsData = remover.transform(wordsDataRaw)

val hashingTF = new HashingTF(10000).setInputCol("words").setOutputCol("rawFeatures")
val featurizedData = hashingTF.transform(wordsData)
featurizedData.show()

Name: Unknown Error
Message: <console>:116: error: type mismatch;
 found   : Int(10000)
 required: String
       val hashingTF = new HashingTF(10000).setInputCol("words").setOutputCol("rawFeatures")
                                     ^

StackTrace: 

In [60]:
val idf = new IDF().setInputCol("rawFeatures").setOutputCol("features")
val idfModel = idf.fit(featurizedData)
val rescaledData = idfModel.transform(featurizedData)

idf = idf_98c6af01e1dc
idfModel = idf_98c6af01e1dc
rescaledData = [id: bigint, name: string ... 5 more fields]


[id: bigint, name: string ... 5 more fields]

In [61]:
object udfUtils extends Serializable {
    def calcNorm(vectorA: SparseVector): Double = {
      var norm = 0.0
      for (i <-  vectorA.indices){ norm += vectorA(i)*vectorA(i) }
      if (norm > 0 ) (math.sqrt(norm)) else (0)
    }
    val calcNormDF = udf[Double,SparseVector](calcNorm)
    
    def cosineSimilarity(vectorA: SparseVector, vectorB:SparseVector,normASqrt:Double,normBSqrt:Double) :(Double) = {
    var dotProduct = 0.0
    for (i <-  vectorA.indices){ dotProduct += vectorA(i) * vectorB(i) }
    val div = (normASqrt * normBSqrt)
    if( div == 0 ) (0)
    else (dotProduct / div)
    }
    
    val calcCosineUDF = udf[Double,SparseVector,SparseVector,Double,Double](cosineSimilarity)
}

defined object udfUtils


In [62]:
val normalized = rescaledData.withColumn("norm", udfUtils.calcNormDF(col("features")))
//normalized.show()

normalized = [id: bigint, name: string ... 6 more fields]


[id: bigint, name: string ... 6 more fields]

In [63]:
val wordsQueryRaw = tokenizer.transform(film)
val wordsQuery = remover.transform(wordsQueryRaw)
val featurizedQuery = hashingTF.transform(wordsQuery)
val rescaledQuery = idfModel.transform(featurizedQuery)
val normalizedQuery = rescaledQuery.withColumn("norm2", udfUtils.calcNormDF(col("features"))).withColumnRenamed("features", "features2").drop("name")
normalizedQuery.show()


+----+----+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+
|lang|  id|                desc|              wordsR|               words|         rawFeatures|           features2|            norm2|
+----+----+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+
|  es|3879| Miguel uno de lo...|[, miguel, uno, d...|[, miguel, uno, d...|(262144,[335,2009...|(262144,[335,2009...|538.8291951140469|
+----+----+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+



wordsQueryRaw = [lang: string, id: bigint ... 3 more fields]
wordsQuery = [lang: string, id: bigint ... 4 more fields]
featurizedQuery = [lang: string, id: bigint ... 5 more fields]
rescaledQuery = [lang: string, id: bigint ... 6 more fields]
normalizedQuery = [lang: string, id: bigint ... 6 more fields]


[lang: string, id: bigint ... 6 more fields]

In [64]:
val cross = normalizedQuery.crossJoin(normalized).drop(normalizedQuery.col("id"))
val cosine = cross.withColumn("similarity", udfUtils.calcCosineUDF(col("features"), col("features2"), col("norm"), col("norm2")))

cross = [lang: string, desc: string ... 13 more fields]
cosine = [lang: string, desc: string ... 14 more fields]


[lang: string, desc: string ... 14 more fields]

In [65]:
val similarsDF = cosine.sort(desc("similarity"),'name,'id).select("id").limit(11).collect().takeRight(10).toList.map(r => r(0))

similarsDF = List(7602, 20593, 8832, 5218, 8727, 7173, 3683, 18884, 7172, 16718)


List(7602, 20593, 8832, 5218, 8727, 7173, 3683, 18884, 7172, 16718)

In [21]:
val ret = Map(23325 -> similarsDF)
import io.circe.syntax._
import io.circe.generic.auto._
println(similarsDF.asJson)
val res = ret :: Nil
//println(res.asJson)

Name: Compile Error
Message: <console>:64: error: object circe is not a member of package io
       import io.circe.syntax._
                 ^
<console>:65: error: object circe is not a member of package io
       import io.circe.generic.auto._
                 ^
<console>:66: error: value asJson is not a member of List[Any]
       println(similarsDF.asJson)
                          ^

StackTrace: 

In [ ]:
spark.stop